In [1]:
#==task4 Wide&Deep==

# 6.1点击率预估简介
# 点击率预估是用来解决什么问题？
# 测试用户是点击还是不点，还是点击的概率，后者称pClick

# 点击率预估模型需要做什么？
# 点击率预估问题就是一个二分类的问题，在机器学习中可以使用逻辑回归作为模型的输出，其输出的就是一个概率值，我们可以将机器学习输出的这个概率值认为是某个用户点击某个广告的概率

# 点击率预估与推荐算法有什么不同？
# 广告点击率预估是需要得到某个用户对某个广告的点击率(喜好),然后结合广告的出价用于排序；
# 而推荐算法只需要得到一个最优的推荐次序，即TopN推荐的问题。当然也可以利用广告的点击率来排序，作为广告的推荐。

# 6.2FM与Wide&Deep呢

# FM模型缺点：当query-item矩阵是稀疏并且是high-rank的时候（比如user有特殊的爱好，或item比较小众），很难非常效率的学习出低维度的表示。
# 这种情况下，大部分的query-item都没有什么关系。但是dense embedding会导致几乎所有的query-item预测值都是非0的，这就导致了推荐过度泛化，会推荐一些不那么相关的物品。
# 简单的linear model却可以通过cross-product transformation（后面讲）来记住这些exception rules。

# 6.3Wide&Deep模型的“记忆能力”与“泛化能力”

# Memorization 和 Generalization是推荐系统很常见的两个概念。
# Memorization指的是通过用户与商品的交互信息矩阵学习规则，而Generalization则是泛化规则。
# FM算法就是很好的Generalization的例子，
# 它可以根据交互信息学习到一个比较短的矩阵V ，其中vi储存着每个用户特征的压缩表示（embedding）;
# 而协同过滤与SVD都是靠记住用户之前与哪些物品发生了交互从而推断出的推荐结果;
# 这两者推荐结果存在一些差异,Wide&Deep模型就能够融合这两种推荐结果做出最终的推荐，得到一个比之前的推荐结果都好的模型。

# Memorization趋向于更加保守，推荐用户之前有过行为的items。
# 相比之下，generalization更加趋向于提高推系统的多样性（diversity）。
# Memorization只需要使用一个线性模型即可实现，而Generalization需要使用DNN实现。

# wide&deep模型的结构图，由左边的wide部分(一个简单的线性模型)，右边的deep部分(一个典型的DNN模型)。

# 实际用的时候要选择哪些特征放在Wide部分，哪些特征放在Deep部分。

# 如何理解Wide部分有利于增强模型的“记忆能力”，Deep部分有利于增强模型的“泛化能力”？
# 1. wide部分是一个广义的线性模型，输入的特征主要有两部分组成，一部分是原始的部分特征，另一部分是原始特征的交互特征(cross-product transformation)，
# 对于wide部分训练时候使用的优化器是带 正则的FTRL算法非常注重模型稀疏性质的，也就是说W&D模型采用L1 FTRL是想让Wide部分变得更加的稀疏，即Wide部分的大部分参数都为0，这就大大压缩了模型权重及特征向量的维度。
# ***Wide部分模型训练完之后留下来的特征都是非常重要的，那么模 型的“记忆能力”就可以理解为发现"直接的"，“暴力的”，“显然的”关联规则的能力。***

# 2. Deep部分是一个DNN模型，输入的特征主要分为两大类，一类是数值特征(可直接输入DNN)，一类是类别特征(需要经过Embedding之后才能输入到DNN中)，
# 我们知道DNN模型随着层数的增加，中间的特征就越抽象，也就提高了模型的泛化能力。
# Deep部分的DNN模型作者使用了深度学习常用的优化器AdaGrad，这也是为了使得模型可以得到更精确的解。

# Wide部分与Deep部分的结合:
# W&D模型是将两部分输出的结果结合起来联合训练，将deep和wide部分的输出重新使用一个逻辑回归模型做最终的预测，输出概率值。

# 6.4操作流程

# 1. Retrieval检索：利用机器学习模型和一些人为定义的规则，来返回最匹配当前Query的一个小的items集合，这个集合就是最终的推荐列表的候选集。
# 2. Ranking排序：
# a. 收集更细致的用户特征，如：
# User features（年龄、性别、语言、民族等）
# Contextual features(上下文特征：设备，时间等)
# Impression features（展示特征：app age、app的历史统计信息等）
# b. 将特征分别传入Wide和Deep一起做训练。

# 训练时，根据最终的loss计算出gradient，反向传播到Wide和Deep两部分中，分别训练自己的参数（wide组件只需要填补deep组件的不足就行了，所以需要比较少的crossproduct feature transformations，而不是full-size wide Model）

# 训练方法是用mini-batch stochastic optimization。
# Wide组件是用FTRL（Follow-the-regularized-leader） + L1正则化学习。
# Deep组件是用AdaGrad来学习。

# c. 训练完之后推荐TopN

# 想要很好的使用wide&deep模型的话，还是要深入理解 业务，确定wide部分使用哪部分特征，deep部分使用哪些特征，以及wide部分的交叉特征应该如何去选择。





In [2]:
# 6.5代码实战
# 代码实践分两部分;
# 1、使用tensorflow已经封装好的wide&deep模型，可以通过它，熟悉模型训练的整体结构；
# 2、使用tensorflow中的keras实现wide&deep，尽可能的看到模型内部的细节并将其实现。


In [ ]:
# ==Tensorflow内置的WideDeepModel==
# ==WideDeepModel==：
tf.keras.experimental.WideDeepModel(
 linear_model, dnn_model, activation=None, **kwargs
) 
# 上面就是将linear_model与dnn_model拼接在了一起，对应于Wide-Deep FM中的最后一步。

In [ ]:
# 将linear_model与dnn_model做一个最简单的实现
# wide部分：直接调用一个keras.experimental中的linear_model线性模型，
linear_model = LinearModel()
#简单实现一个全连接神经网络
dnn_model = keras.Sequential([keras.layers.Dense(units=64),
 keras.layers.Dense(units=1)])
# 使用WideDeepModel将前两步产生的两个model拼接在一起，形成最终的combined_model，
combined_model = WideDeepModel(linear_model, dnn_model)
#compile赋参
combined_model.compile(optimizer=['sgd', 'adam'], 'mse', ['mse'])
# define dnn_inputs and linear_inputs as separate numpy arrays or
# a single numpy array if dnn_inputs is same as linear_inputs.
# fit训练
combined_model.fit([linear_inputs, dnn_inputs], y, epochs)
# or define a single `tf.data.Dataset` that contains a single tensor or
# separate tensors for dnn_inputs and linear_inputs.
dataset = tf.data.Dataset.from_tensors(([linear_inputs, dnn_inputs], y))
combined_model.fit(dataset, epochs)


In [ ]:
# 线性模型与DNN模型在联合训练之前均可进行分别训练，如下：
#训练了一个线性模型
linear_model = LinearModel()
linear_model.compile('adagrad', 'mse')
linear_model.fit(linear_inputs, y, epochs)
# 训练DNN模型
dnn_model = keras.Sequential([keras.layers.Dense(units=1)])
dnn_model.compile('rmsprop', 'mse')
dnn_model.fit(dnn_inputs, y, epochs)
#两个模型联合训练
combined_model = WideDeepModel(linear_model, dnn_model)
combined_model.compile(optimizer=['sgd', 'adam'], 'mse', ['mse'])
combined_model.fit([linear_inputs, dnn_inputs], y, epochs)


In [4]:
# ==Tensorflow keras 实现wide&deep模型==
# 注意以下问题：
# 特征操作：对原始特征进行转换，deep特征和wide特征的选择，特征的交叉，
# 也分成wide部分和deep部分，
# 为了简单实现，使用了同一个优化器优化两部分。
# 代码 待更




In [ ]:
# 6.6深度学习推荐系统的发展
# wide改进，Deep改进，发展出其它模型
# ……

# 6.7课后思考
# 思考不足和改进

# 6.8参考资料
# Google 的 Wide&Deep
# CTR
# Numpy手工打造Wide&Deep